In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
import analyze_utils

save_dir =  '/home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/'

r, all_losses = analyze_utils.load_results_and_cache_autoprompt_json(
    save_dir, include_losses=True, do_reranking=False, save_file='r.pkl')

getting dir_names...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 134/134 [00:01<00:00, 82.76it/s]

skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/Jan_25_08_00_affblcamvbok/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/Jan_25_08_06_auykallxfffo/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/R-interpretable-autoprompting.259718.out/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/R-interpretable-autoprompting.259719.out/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_icml/classification__prefix_after_input_2/R-interpretable-autoprompting.259720.out/results.pkl (pkl still writing?)
skipping /home/jxm3/research/prompting/interpretable-autoprompting/results_

In [4]:
len(r)

30068

In [5]:
pd.set_option('display.max_colwidth', None)

top_prompts = (
    r 
      # .sort_values(by='prefix_train_acc', ascending=False)
      .sort_values(by='prefix_train_loss', ascending=True)
      .groupby(['num_learned_tokens', 'task_name', 'model_cls', 'seed'])
    
).first()
print(len(top_prompts))

top_prompts[['prefixes', 'reciprocal_rank', 'prefix_train_loss', 'prefix_train_acc']]

59


prefixes  \
num_learned_tokens task_name  model_cls  seed                                                                                                                                              
6                  ffb_train  autoprompt 1                                                                                                                EFFverbal EUR Thorntonshopnown   
                                         2                                                                                                             Fur resultolandgroundur augmented   
                                         3                                                                                                                      Hackmmmmajoreryitprofits   
                              iprompt    1                                                                                                                    almost neutral. However, "   
                                         2                                                                                                                              "So, a bottle of   
                                         3                                                                                                                 "Does this represent a market   
                   imdb_train autoprompt 1                                                                                                             CRIP deserves PIN SOC sling level   
                                         2                                                                                                                    as ​Overall': large points   
                                         3                                                                                               ™:Supplement Reasons****************RatingUltra   
                              iprompt    1                                                                                                                 When you watch and enjoy this   
                                         2                                                                                                                      I find this film a total   
                                         3                                                                                                                   To summarize this review! :   
                   rt_train   autoprompt 1                                                                                                         Whether{{ anotherath<|endoftext|> how   
                                         2                                                                                                    congratulations Named #SPONSOREDReport the   
                                         3                                                                                                                     wow some oneendered  very   
                              iprompt    1                                                                                                                  "not only are the characters   
                                         2                                                                                                                    Who is the author of these   
                                         3                                                                                                                   Do you agree with the above   
                   sst2_train autoprompt 1                                                                                                         \t BryceSpecificallyWASHINGTONRatedam   
                                         2                                                                                                                396 trulyCustomer echoes the "   
                                        

In [6]:
top_prompts.groupby(['task_name', 'model_cls']).mean()['prefix_train_acc']

task_name   model_cls 
ffb_train   autoprompt    0.692708
            iprompt       0.804688
imdb_train  autoprompt    0.888021
            iprompt       0.915179
rt_train    autoprompt    0.772321
            iprompt       0.815972
sst2_train  autoprompt    0.809028
            iprompt       0.862847
Name: prefix_train_acc, dtype: float64

In [7]:
assert r['checkpoint'].unique()[0] == "EleutherAI/gpt-j-6B"

from iprompt import prompt_classification

model = prompt_classification.create_model(r['checkpoint'].unique()[0])

2023-01-25 12:00:11.139539: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-25 12:00:11.307126: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-25 12:00:11.341634: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-25 12:00:12.545495: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [39]:
## Compute accuracy given correct prompt and save for each task.
import argparse
from tqdm.notebook import tqdm
from iprompt.data import get_data


data = []
print('calculating accs...')
n_shots = 1
batch_size = 16

"""
task_name: str = 'add_two',
 n_shots: int = 1,
 train_split_frac: float = None,
 max_dset_size: int = 10000,
 template_num_task_phrasing: int = 0,
 max_digit: int = 10,
 """

for _, output in tqdm(top_prompts.reset_index().iterrows(), total=len(top_prompts)):
    verbose = False
    max_length = 128
    if not (('ffb' in output['task_name']) or ('imdb' in output['task_name'])): continue
    if output['model_cls'] == 'autoprompt': continue
    # if 'ffb' in output['task_name']: continue
    output['task_name'] = output['task_name'].replace('_train', '_test')
    args = argparse.Namespace(**output)
    args.train_split_frac = 1.0 # take 100% of test set
    args.max_dset_size = 1_000
    print("*-*-" * 20)
    print(args.task_name)
    (dset, __dset_test), check_answer_func, descr = get_data(
        args.task_name, n_shots=n_shots, train_split_frac=args.train_split_frac,
        max_dset_size=args.max_dset_size, template_num_task_phrasing=0,
    )
    # if task_name == 'task107_splash_question_to_sql':
    #     batch_size = max(1, batch_size//4)
    ####   Manual prompt  ####
    descr = "" # tmp override
    # manual_loss, manual_acc = prompt_classification.test_model_on_task_with_prefix(
    #     dset=dset, model=model, prefix=descr, multi_token=False, verbose=verbose,
    #     max_length=max_length, batch_size=64, tqdm_notebook=True,
    #     restrict_to_valid_answers=True,
    #     prefix_before_input=False,
    # )
    # print(output)
    # print(f'\t{descr} || {manual_acc:.1f}%')
    ####   iPrompt prompt   ####
    iprompt_loss, iprompt_acc = prompt_classification.test_model_on_task_with_prefix(
        dset=dset, model=model, prefix=output['prefixes'], multi_token=False, verbose=verbose,
        max_length=max_length, batch_size=64, tqdm_notebook=True,
        restrict_to_valid_answers=True,
        prefix_before_input=False,
    )
    print(f'\t{output["prefixes"]} || {iprompt_acc:.1f}%')
    ####
    output['manual_acc'] = manual_acc
    output['iprompt_acc'] = iprompt_acc
    data.append(output)

calculating accs...


  0%|          | 0/59 [00:00<?, ?it/s]

*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
ffb_test
**loading data: financial_phrasebank // train


Found cached dataset financial_phrasebank (/home/jxm3/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)
Loading cached shuffled indices for dataset at /home/jxm3/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141/cache-5e700914242e73b3.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1698 [00:00<?, ?ex/s]

  0%|          | 0/16 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 132.00 MiB (GPU 0; 23.65 GiB total capacity; 21.18 GiB already allocated; 65.44 MiB free; 22.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
[[4]] * 5

[[4], [4], [4], [4], [4]]

In [17]:
df_with_acc = pd.DataFrame(data)
# df_with_acc[['task_name', 'model_cls', 'seed', 'prefixes', 'prefix_train_acc', 'iprompt_acc', 'manual_acc']]
df_with_acc.groupby(['num_learned_tokens', 'task_name', 'model_cls']).mean()[['prefix_train_acc', 'iprompt_acc', 'manual_acc']]

prefix_train_acc  iprompt_acc  \
num_learned_tokens task_name  model_cls                                   
6                  ffb_train  autoprompt          0.718750    70.020000   
                              iprompt             0.817708    70.166667   
                   imdb_train autoprompt          0.864583    59.266667   
                              iprompt             0.895833    60.133333   
                   rt_train   autoprompt          0.812500    71.733333   
                              iprompt             0.807292    83.966667   
                   sst2_train autoprompt          0.838542    82.866667   
                              iprompt             0.859375    86.333333   
12                 ffb_train  autoprompt          0.666667    69.366667   
                              iprompt             0.791667    78.800000   
                   imdb_train autoprompt          0.911458    60.033333   
                              iprompt             0.921875    59.966667   
                   rt_train   autoprompt          0.750000    75.100000   
                              iprompt             0.848958    84.100000   
                   sst2_train autoprompt          0.833333    65.266667   
                              iprompt             0.880208    86.500000   
24                 imdb_train iprompt             0.953125    60.000000   
                   rt_train   autoprompt          0.718750    51.600000   
                              iprompt             0.791667    80.300000   
                   sst2_train autoprompt          0.755208    67.600000   
                              iprompt             0.848958    86.266667   

                                          manual_acc  
num_learned_tokens task_name  model_cls               
6                  ffb_train  autoprompt   47.100000  
                              iprompt      47.100000  
                   imdb_train autoprompt   58.833333  
                              iprompt      59.800000  
                   rt_train   autoprompt   60.600000  
                              iprompt      60.600000  
                   sst2_train autoprompt   64.200000  
                              iprompt      64.200000  
12                 ffb_train  autoprompt   47.100000  
                              iprompt      47.100000  
                   imdb_train autoprompt   59.800000  
                              iprompt      59.800000  
                   rt_train   autoprompt   60.600000  
                              iprompt      60.600000  
                   sst2_train autoprompt   64.200000  
                              iprompt      64.200000  
24                 imdb_train iprompt      59.800000  
                   rt_train   autoprompt   60.600000  
                              iprompt      60.600000  
                   sst2_train autoprompt   64.200000  
                              iprompt      64.200000

## Loading with PromptSource

In [ ]:
import promptsource
import promptsource.templates

imdb_prompts = promptsource.templates.DatasetTemplates('imdb')

pos_input = { "text": "\"What a wonderful film :) \"", "label": 1 }
neg_input = { "text": "\"This movie sucks!\"", "label": 0 }

for tn in imdb_prompts.all_template_names:
    print(tn)
    print('\t [+]', imdb_prompts[tn].apply(pos_input))
    print('\t [-]', imdb_prompts[tn].apply(neg_input))
    print()

In [ ]:
imdb_prompts["Movie Expressed Sentiment"].apply({ "text": "This movie sucks!" })